In [71]:
import sys , os 
import numpy as np
import math as m
import pandas as pd

import plotly.offline as py

import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)

In [89]:
Plot1="/home/crct/CRCT/JupyterNotebook/gammaIndex/test.csv"
Plot2="/home/crct/CRCT/JupyterNotebook/gammaIndex/test2.csv"


mesurePlotData=pd.read_csv(Plot1, sep=';')
gatePlotData=pd.read_csv(Plot2, sep=';')

In [103]:
LinearInterpolation=False
#Point every (in mm)
#NewStep=0.1

Threshold=True
#Of Dmax
ThresholdValue=0.1

#Local or Global
Global=1

#Gamma Index criterion 
DTA=1 #mm
deltaDmax=1 #%

In [55]:
#pour scaling en x
#xGscale=0.0

In [104]:
#Create numpy ndarray from pandas dataFrames
refX=mesurePlotData['step']  # mettre le header qu'il y a dans les csv 
refY=mesurePlotData['value'] # mettre le header qu'il y a dans les csv 
gateX=gatePlotData['step'] # mettre le header qu'il y a dans les csv 
gateY=gatePlotData['value'] # mettre le header qu'il y a dans les csv 

# Compute Linear interpolation for simulated values and measured values according to your parameters (change 1000 values to change number of points)
if LinearInterpolation==True:
    refX2=np.linspace(refX.min(), refX.max(), 2000)
    refY2=np.interp(refX2, refX, refY)
    gateX2=np.linspace(gateX.min(), gateX.max(), 2000)
    gateY2=np.interp(gateX2, gateX, gateY)


else:
    refX2= refX
    refY2= refY
    gateX2= gateX
    gateY2= gateY



    #SCALING en X
#gateX2=gateX2+xGscale

print 'ok'

ok


In [106]:
#  GATE  #
#create numpy ndarray for gamma comprison
percentDmax1=((deltaDmax/100.0)*gateY2.max())
print gateY2.max()
deltaD=np.zeros(len(refY2))
gammaIndex=np.zeros(len(gateX2))
gammaIndexTEMP=np.zeros(len(refX2))
drTEMP=np.zeros(len(refX2))
dr=np.zeros(len(refX2))

#compute Gamma Index 1D
for i in range(len(gateX2)):
    for j in range(len(refX2)):
        dr[j]=(gateX2[i]-refX2[j])
        deltaD[j]=gateY2[i]-refY2[j]
        if Global == 1:
            gammaIndexTEMP[j]=m.sqrt(((deltaD[j]**2)/percentDmax1**2) + ((dr[j]**2)/DTA**2))
        else:
            gammaIndexTEMP[j]=m.sqrt(((deltaD[j]**2)/((deltaDmax/100.0)*refY2[j])**2) + ((dr[j]**2)/DTA**2))
    gammaIndex[i]=gammaIndexTEMP.min()

#Compute Gamma Index Pass Rate (with threshold or not)
nok=0.0
a=0
if Threshold==True: 
    for i in range(len(gammaIndex)):

        if gateY2[i]>0.10:
            a=a+1
            if gammaIndex[i]<1:
                nok=nok+1
    print ('nombre de points testés : ' +str(a))
    print ('nombre de points passant le test (<1) : ' +str(nok))
    passRate=(nok/a)*100
else:
    nok=(gammaIndex < 1).sum()
    passRate=nok*100.0/len(gammaIndex)
#print results
percentDmax=deltaDmax*100
print('RESULTAT GATE : ')
print('*   Gamma Index Pass Rate '+str(percentDmax) + '%-' +str(DTA) + 'mm: '  +str(passRate) +' %')

100.0
nombre de points testés : 240
nombre de points passant le test (<1) : 180.0
RESULTAT GATE : 
*   Gamma Index Pass Rate 100%-1mm: 75.0 %


In [107]:
# Plot profil or PDD with plotly      
trace1 = go.Scatter(
    x = refX2,
    y = refY2,
    mode = 'lines', 
    name = 'Film Measure', 
    
    line = dict(
        color = ('rgba(214, 39, 40, 100)')
    )
)

trace2 = go.Scatter(
    x = gateX2,
    y = gateY2,
    mode = 'lines', 
    name = 'GATE', 
    line = dict(
        color = ('rgba(44, 160, 44, 100)'),
        dash='dot', 
        width=3
    ),
#     error_y=dict(
#         type='data',
#         array=gateUy2,
#         visible=True,
#         thickness=0.5,
#         width=1.5,
#         opacity=0.5,
#         color='#9CEF9C'
#     )
)

    #Plot gamma Index with plotly
trace4 = go.Scatter(
    x = gateX2,
    y = gammaIndex,
    mode = 'lines', 
    name = 'Gamma index GATE', 
    line = dict(
        color = ('rgba(44, 160, 44, 100)'),
        dash='dot', 
        width=3
    )
)

print 'ok'

ok


In [111]:
val1=str(round(passRate,2))

layout1= go.Layout(
    title='Dose MeV',
    titlefont=dict(size=24),
    xaxis=dict(
        range=[0, 330],
        title='distance (mm)',
        titlefont=dict(
            size=20
        )
),      
    yaxis=dict(
        title='Relative Dose',
        titlefont=dict(
        size=20)
    )
)


layout2= go.Layout(
    title='Gamma Index',
    titlefont=dict(size=24),
    xaxis=dict(
        range=[0, 330],
        title='distance (mm)',
        titlefont=dict(
            size=20
        )
),      
    yaxis=dict(
        title='Relative Dose',
        titlefont=dict(
        size=20)
    )
)


print 'ok'

ok


In [112]:
data1 = [trace1, trace2]
fig1= go.Figure(data=data1, layout=layout1) 

data2 = [trace4]
fig2= go.Figure(data=data2, layout=layout2)

py.iplot(fig1, filename='plot', image='svg') 
py.iplot(fig2, filename='gamma', image='svg') 